In [1]:
import sys
sys.path.append("../src")

import pandas as pd


import torch
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

from fuzzywuzzy import fuzz
from tqdm import tqdm
import re
from preprocess import Preprocessing

device = 'cuda' if torch.cuda.is_available() else 'cpu'

/Users/SamuelLP/Desktop/git/projet_nlp/.nlp_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/SamuelLP/Desktop/git/projet_nlp/.nlp_venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# import datas and clean text

In [3]:
train_df = pd.read_csv("../datas/train_data.csv")

In [4]:
def remove_newlines(df):
    df = df.replace("\n", '', regex=True)
    return df
train_df = remove_newlines(train_df)

preprocess = Preprocessing(train_df)
train_df = preprocess.remove_stopwords()

train_df.head()

,ID,filename,texte,sexe,date_accident,date_consolidation
0,0,Agen_100515.txt,Le : 12/11/2019 Cour d’appel d’Agen chambre so...,homme,1991-04-09,n.c.
1,1,Agen_1100752.txt,Le : 12/11/2019 Cour d’appel d’Agen chambre ci...,homme,2005-06-10,2010-01-19
2,2,Agen_1613.txt,Le : 12/11/2019 Cour d’appel d’Agen Audience p...,femme,1997-09-26,n.c.
3,3,Agen_2118.txt,Le : 12/11/2019 Cour d’appel d’Agen Audience p...,femme,1982-08-07,1982-11-07
4,4,Agen_21229.txt,Le : 12/11/2019 Cour d’appel d’Agen Audience p...,homme,1996-11-26,n.c.


# Text batching and apply the NER model

In [4]:
def divide_text_into_batches(text, n_batches):
    """
    Divise the text on n batches.
    """
    batch_size = len(text) // n_batches
    return [text[i:i+batch_size] for i in range(0, len(text), batch_size)]

def extract_sentences_around_dates(batch, model, tokenizer, context_window):
    """
    extract dates and a context of N tokens before and after the dates in a batch
    """
    ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)
    results = ner_pipeline(batch)
    dates_and_contexts = []

    for result in results:
        if result['entity'] == 'B-DATE' or result['entity'] == 'I-DATE':
            start, end = result['start'], result['end']
            pre_context = batch[max(0, start - context_window*6):start]
            post_context = batch[end:end + context_window*6]
            
            # Extract the the text around the date
            
            # before the date
            sentence_boundaries = re.search(r'([.!?]\s+)|([.!?]$)', pre_context[::-1])
            pre_sentence_boundary = -sentence_boundaries.start(0) if sentence_boundaries else -len(pre_context)
            
            # after the date
            sentence_boundaries = re.search(r'([.!?]\s+)|([.!?]$)', post_context)
            post_sentence_boundary = sentence_boundaries.end(0) if sentence_boundaries else len(post_context)
            
            full_context = pre_context[pre_sentence_boundary:] + batch[start:end] + post_context[:post_sentence_boundary]
            
            # Clean and extract words around the date
            words_around_date = re.findall(r'\w+', full_context)
            date_index = len(re.findall(r'\w+', pre_context[pre_sentence_boundary:]))
            start_context = max(0, date_index - context_window)
            end_context = min(len(words_around_date), date_index + context_window)
            
            context_sentence = ' '.join(words_around_date[start_context:end_context])
            dates_and_contexts.append((batch[start:end], context_sentence))

    return dates_and_contexts

def extract_dates_with_context_from_long_text(text, model, tokenizer, n_batches, context_window):
    """
    Handles long texts by dividing them into batches and extracting sentences around identified dates.
    """
    batches = divide_text_into_batches(text, n_batches)
    all_dates_and_contexts = []

    for batch in batches:
        batch_dates_and_contexts = extract_sentences_around_dates(batch, model, tokenizer, context_window)
        all_dates_and_contexts.extend(batch_dates_and_contexts)

    return all_dates_and_contexts


# Apply the function to all the rows of the dataframe

In [5]:
MODEL_NAME = "Jean-Baptiste/camembert-ner-with-dates"
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
N_BATCHES = 15
CONTEXT_WINDOW = 50

In [6]:
data_to_append = []

for index, row in tqdm(train_df.iterrows(), total=train_df.shape[0]):
    text = row["texte"]
    filename = row["filename"]
    
    dates_and_contexts = extract_dates_with_context_from_long_text(text, model, tokenizer,
                                                                   N_BATCHES, CONTEXT_WINDOW)
    
    for _, context in dates_and_contexts:
        data_to_append.append({"filename": filename, "texte": context})
    
results_df = pd.DataFrame(data_to_append, columns=["filename", "texte"])


100%|██████████| 770/770 [7:43:30<00:00, 36.12s/it]      


# Let's use fuzzy wuzzy to remove the duplicated rows

In [7]:
def non_duplicate_rows(df, threshold):
    rows_to_keep = []

    for i in range(len(df) - 1):
        ratio = fuzz.ratio(df.iloc[i]['texte'], df.iloc[i+1]['texte'])
        if ratio < threshold:
            rows_to_keep.append(i)
    
    rows_to_keep.append(len(df) - 1)
    
    return df.iloc[rows_to_keep]

THRESHOLD = 95
df_fuzz = non_duplicate_rows(results_df, THRESHOLD)

In [8]:
df_fuzz.reset_index(drop=True, inplace=True)

In [11]:
df_fuzz = df_fuzz.groupby('filename')['texte'].apply(' '.join).reset_index()

In [12]:
df_fuzz.to_csv("../datas/passages_dates.csv", index=False)